In [25]:
import datetime
import hmac
import time
from base64 import b64encode

import requests

class RequestError(Exception):
    pass


class Auth():
    '''PTX authentication  
    
    '''
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key
    
    def signature(self, xdate):
        HMAC = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), 'sha1').digest()
        return b64encode(HMAC).decode()

    def request(self, endpoint, params, **kwargs):
        api = 'https://ptx.transportdata.tw/MOTC'
        xdate = time.strftime("%a, %d %b %Y %I:%M:%S GMT", time.gmtime())
        signature = self.signature(xdate)
        headers = {
            'Authorization': f'hmac username="{self.app_id}", algorithm="hmac-sha1", headers="x-date", signature="{signature}"',
            'x-date': xdate,
            'Accept-Encoding': 'gzip, deflate'
        }
        with requests.get(f'{api}{endpoint}', headers=headers, params=params, **kwargs) as req:
            if req.status_code != 200:
                raise RequestError(f'請求錯誤 {req.status_code}')
            return req.content

In [26]:
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64
from requests import request
from pprint import pprint

app_id = 'citymapper_tw'
app_key = 'WBK.T27F93'

class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }


if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/Stop/City/Taipei?$top=30&$format=JSON', headers= a.get_auth_header())
    pprint(response.content)

b'{"message":"HMAC signature cannot be verified"}\n'
